# Code for the simulation of the behaviour of the two populations
The considered population are rabbits and wolves that move in the same space (forest) and behave according to the Predator-Prey model.
The visualization of the simulation consists of two parts:
<ol>
<li> <b>Animation - </b> implemented using the vpython package, that represents the way the individuals move in the forest </li>
<li> <b>Animated graphs - </b>   that represent the changes in the population size in time</li>

</ol>
Both animation and graphs are synchronized. <br>
The implementation is object-oriented. 3 classes are created:
<ol>
<li> <b>Rabbit </b>(Pardwa) </li>
<li> <b>Wolf </b>(Wilk)  </li> 
<li> <b>Forest </b>(Las)  </li> 
</ol>

The created GUI merges these classes, animation and graphs.
The simulation can be turned on by the GUI that requires to specify the values of the interaction parameters. 



In [1]:
from vpython import *
import numpy.random as rn
import math
from tkinter import ttk
from tkinter import *
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import numpy as np

<IPython.core.display.Javascript object>

In [2]:
class pardwa:
    def __init__(self,plec,a,b,r,sigma):
        self.pos=vec(a,b,0)
        self.plec=plec
        self.r=r
        if self.plec=='f':
            self.color=vec(0.7,0.4,1)
        else:
            self.color=vec(0.49,0,1)
        self.p=sphere(make_trail=False,pos=vec(a,b,0), radius=self.r, color=self.color)
        self.sigma=sigma
        self.wiek=0
        self.czas_zycia=rn.exponential(70)
        self.skladanie_jaj=False
        self.dojrzala=False
        self.przemieszczenie=False
        
    def brown(self):
        self.p.pos=self.p.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)
        self.p.trail=curve(color=self.color)
        
    def blisko_samce(self,x_samca2,y_samca2,d):
        self.p.pos=self.p.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)-3/(d+0.01)*(vec(x_samca2,y_samca2,0)-self.p.pos)
        self.przemieszczenie=True

    def gody_samica(self,x_samca,y_samca):
        self.p.pos=self.p.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)+(vec(x_samca,y_samca,0)-self.p.pos)
        self.przemieszczenie=True
        
    def gody_samiec(self,x_samica,y_samica):
        self.p.pos=self.p.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)+1.5*(vec(x_samica,y_samica,0)-self.p.pos)
        self.przemieszczenie=True

    def po_godach(self,x_drugiego,y_drugiego):
         self.p.pos=self.p.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)-3*(vec(x_drugiego,y_drugiego,0)-self.p.pos)
         self.przemieszczenie=True

    def ile_jaj_bedzie(self):
        return rn.poisson(40,1)
                        
    def smierc(self):
        if self.wiek>=self.czas_zycia:
            self.p.visible=False
            return True
        else:
            self.wiek+=1
            return False
    def dojrzalosc(self):
        self.dojrzala=True

    def kasuj_przemieszczenia(self):
        self.przemieszczenie=False

    def nie_przemieszczona_p_przemiescic(self):
        if self.przemieszczenie==False:
            self.brown()
            self.przemieszczenie=True

    def uciekaj_przed_wilkiem(self,w_x,w_y):
        self.p.pos=self.p.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)+0.9*(vec(w_x,w_y,0)-self.p.pos)
        self.przemieszczenie=True

In [3]:
class wilk:
    def __init__(self,plec,a,b,r,sigma,na_glodzie):
        alfa=rn.uniform(0,1)
        if alfa<=0.05:
            self.alfa=True
        else:
            self.alfa=False
        self.pos=vec(a,b,0)
        self.plec=plec
        self.r=r
        if self.plec=='f':
            self.color=vec(0.75,0.75,0.75)
        else:
            self.color=color.black
        self.wiek=0
        self.czas_zycia=rn.exponential(200)
        self.sigma=sigma
        self.w=sphere(make_trail=False,pos=vec(a,b,0), radius=self.r, color=self.color)
        self.przemieszczenie=False
        self.ciaza=False
        self.dojrzala=False
        #wilk.najedzony=False
        self.ile_nie_jadl=0
        self.na_glodzie=na_glodzie
        self.mama=[]
        self.najedzony=False
    def brown(self):
        self.w.pos=self.w.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)
        self.w.trail=curve(color=self.color)
        
    def smierc(self):
        if self.wiek>=self.czas_zycia or self.ile_nie_jadl>self.na_glodzie:
            self.w.visible=False
            return True
        else:
            self.ile_nie_jadl+=1
            self.wiek+=1
            return False

    def kasuj_przemieszczenia(self):
        self.przemieszczenie=False

    def nie_przemieszczona_w_przemiescic(self):
        if self.przemieszczenie==False:
            self.brown()
            self.przemieszczenie=True

    def trzymaj_sie_stada(self,x_drugiego,y_drugiego,d):
        if self.alfa==True:
            pass
        else:
            self.w.pos=self.w.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)+0.1*(vec(x_drugiego,y_drugiego,0)-self.w.pos)*(d-0.1)
            self.przemieszczenie=True

    def dazy_do_alfa(self,x_alfa,y_alfa):
        self.w.pos=self.w.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)+0.15*(vec(x_alfa,y_alfa,0)-self.w.pos)
        self.przemieszczenie=True

    def drugi_alfa_blisko(self,x,y):
        self.w.pos=self.w.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)-(vec(x,y,0)-self.w.pos)
        self.przemieszczenie=True

    def gody_wilkow(self,x,y):
        self.w.pos=self.w.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)+0.3*(vec(x,y,0)-self.w.pos)
        self.przemieszczenie=True

    def po_godach(self,x,y):
        self.w.pos=self.w.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)-2*(vec(x,y,0)-self.w.pos)
        self.przemieszczenie=True

    def ile_wilkow_bedzie(self):
        return rn.poisson(2)

    def dojrzalosc(self):
        self.dojrzala=True

    def mlode_trzymaj_sie_mamy(self,x,y):
        self.w.pos=self.w.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)+1.3*(vec(x,y,0)-self.w.pos)
        self.przemieszczenie=True

    def zjadl(self):
        self.ile_nie_jadl=0
        self.najedzony=True

    def gon_pardwe(self,x,y,d):
        self.w.pos=self.w.pos+vec(rn.normal(0,self.sigma),rn.normal(0,self.sigma),0)+(vec(x,y,0)-self.w.pos)*(1+min(0.3,d**2))

    def sprawdz_czy_najedzony(self):
        if 2*self.ile_nie_jadl==self.na_glodzie:
            self.najedzony=False


In [35]:
class las:
    def __init__(self,pf,pm,wf,wm,R,r,sigma_p,sigma_w,hpmm,hpfm,hpw,hwmm,hwfm,hwalfa,hbreed,na_glodzie):
        self.hpmm=hpmm
        self.hpfm=hpfm
        self.hpw=hpw
        self.hwmm=hwmm
        self.hwfm=hwfm
        self.hbreed=hbreed
        self.na_glodzie=na_glodzie
        
        self.hwalfa=hwalfa
        self.pf=[]
        self.pm=[]
        self.wm=[]
        self.wf=[]
        
        self.po_godach_pardw=[]
        self.skladajace_jaja=[]
        self.zlozone_jaja=[]
        self.niedojrzale_pardwy=[]
        
        self.po_godach_wilkow=[]
        self.w_ciazy=[]
        self.wilki_w_brzuchu=[]
        self.niedojrzale_wilki=[]
        
        self.sigma_p=sigma_p
        self.sigma_w=sigma_w
        
        scene = canvas(title='Forest',
             x=0, y=0, width=800, height=800,
             center=vec(0,0,0), background=vec(0.6,1,0.6))
        self.R=R
        self.r=r
        
        for i in range(0,pf):
            u=rn.uniform(0,2*pi)
            rr=rn.uniform(0,self.R)
            a=math.sqrt(rr)*math.sin(u)
            b=math.sqrt(rr)*math.cos(u)
            p=pardwa('f',a,b,self.r,self.sigma_p)
            self.pf.append(p)
            self.niedojrzale_pardwy.append(p)
        for i in range(0,pm):
            u=rn.uniform(0,2*pi)
            rr=rn.uniform(0,self.R)
            a=math.sqrt(rr)*math.sin(u)
            b=math.sqrt(rr)*math.cos(u)
            p=pardwa('m',a,b,self.r,self.sigma_p)
            self.pm.append(p)
            self.niedojrzale_pardwy.append(p)
        for i in range(0,wm):
            u=rn.uniform(0,2*pi)
            rr=rn.uniform(0,self.R)
            a=math.sqrt(rr)*math.sin(u)
            b=math.sqrt(rr)*math.cos(u)
            w=wilk('m',a,b,self.r,self.sigma_w,self.na_glodzie)
            self.wm.append(w)
            self.niedojrzale_wilki.append([w,False])
        for i in range(0,wf):
            u=rn.uniform(0,2*pi)
            rr=rn.uniform(0,self.R)
            a=math.sqrt(rr)*math.sin(u)
            b=math.sqrt(rr)*math.cos(u)
            w=wilk('f',a,b,self.r,self.sigma_w,self.na_glodzie)
            self.wf.append(w)
            self.niedojrzale_wilki.append([w,False])
        self.lpf=len(self.pf)
        self.lpm=len(self.pm)
        self.lwm=len(self.wm)
        self.lwf=len(self.wf)

        
        
    def run(self):
        #print('iteracja',self.skladajace_jaja)
        self.gody_pardw()
        self.po_godach_pardw_funkcja()
        self.skladanie_jaj_pardw()
        self.jaja_pardw_rosna()
        self.przemieszczanie_samcow_pardw()
        self.sprawdz_dojrzalosc_pardw()
        self.pardwy_uciekaja()

        self.gon_pardwy()
        self.wilki_w_stadzie()
        self.wilki_daza_do_alfy()
        #self.alfa_oddalaja_sie_od_siebie()
        self.gody_wilkow()
        self.po_godach_wilkow_funkcja()
        self.wilki_do_brzucha()
        self.narodziny_wilkow()
        self.sprawdz_dojrzalosc_wilkow()
        self.zauktualizuj_czy_wilki_sa_glodne()

        self.nie_przemieszczone_przemiescic()
        self.starzenie()
        self.kasuj_przemieszczenia()
        #print(' po iteracja',self.skladajace_jaja)
        rate(100)
        
        
    
        
    def przemieszczanie_samcow_pardw(self):
        macierz_spr_pardw=np.zeros((len(self.pm),len(self.pm))) #zeby dwa razy nie sprawdzac tych samych pardw w jednej jednsotce czasu
        for i in range(0,len(self.pm)): #odleglosci miedzy samcami pardw
            for j in range(0,len(self.pm)):
                if i==j:
                    pass 
                else:
                    if macierz_spr_pardw[i][j]==0 and macierz_spr_pardw[j][i]==0:
                        d=(self.pm[i].p.pos.x-self.pm[j].p.pos.x)**2+(self.pm[i].pos.y-self.pm[j].p.pos.y)**2
                        d=math.sqrt(d)
                        if d<self.hpmm:
                            self.pm[i].blisko_samce(self.pm[j].p.pos.x,self.pm[j].p.pos.y,d)

                            macierz_spr_pardw[i][j]=1
                            macierz_spr_pardw[j][i]=1
                            
                                
    def gody_pardw(self):

        for i in range(0,len(self.pm)):
            if self.pm[i].dojrzala==True:
                
                for j in range(0,len(self.pf)):
                    if self.pf[j].dojrzala==True:

                        if self.pf[j].skladanie_jaj==True:
                            pass
                
                        else:
                
                            d=(self.pm[i].p.pos.x-self.pf[j].p.pos.x)**2+(self.pm[i].pos.y-self.pf[j].p.pos.y)**2
                            d=math.sqrt(d)
                
                            if d<self.hpfm/10:
                                self.pf[j].po_godach(self.pm[i].p.pos.x,self.pm[i].p.pos.y)
                                self.pm[i].po_godach(self.pf[j].p.pos.x,self.pf[j].p.pos.y)
                                self.po_godach_pardw.append([self.pm[i],self.pf[j],6]) #tu bedziemy zapisywac pardwy po godach ktore przez 5 jednostek maja sie oddalac
                    
                            elif d<self.hpfm/3:
                                self.pm[i].gody_samiec(self.pf[j].p.pos.x,self.pf[j].p.pos.y)

                            else:
                                if d<=self.hpfm: #samica będzie wabiona
                                    self.pf[j].gody_samica(self.pm[i].p.pos.x,self.pm[i].p.pos.y)


    def po_godach_pardw_funkcja(self):
        for i in self.po_godach_pardw:
            if i[2]>0:
                i[0].po_godach(i[1].p.pos.x,i[1].p.pos.y)
                i[1].po_godach(i[0].p.pos.x,i[0].p.pos.y)
                i[2]=i[2]-1
                                                
            if i[2]==0:
                i[1].składanie_jaj=True
                #print('sklada jaja')
                self.skladajace_jaja.append([i[1],False])
                self.po_godach_pardw.remove(i)

    def skladanie_jaj_pardw(self):
        for i in self.skladajace_jaja[:]:
            
            if i[1]==False:
                i[1]=i[0].ile_jaj_bedzie()
                #print('ile jaj',i[1])
             #wiemy ile juz jaj zlozy
            if i[1]>0:
                self.zlozone_jaja.append([0,vec(i[0].p.pos.x,i[0].p.pos.y,0)])
                i[1]=i[1]-1
            if i[1]==0:
                i[0].skladajnie_jaj=False
                self.skladajace_jaja.remove(i)

    def jaja_pardw_rosna(self):
    
        for i in self.zlozone_jaja[:]:
             if i[0]==4:
                plec_pardwy=rn.uniform(0,1)
                polozenie=i[1]
                if plec_pardwy<=0.5:
                    plec_pardwy='m'
                    p=pardwa('m',polozenie.x,polozenie.y,self.r,self.sigma_p)
                    p.p.color=vec(0,1,0)
                    p.p.visible=True
                    self.pm.append(p)
                    self.niedojrzale_pardwy.append(p)
                else:
                    plec_pardwy='f'
                    p=pardwa('f',polozenie.x,polozenie.y,self.r,self.sigma_p)
                    p=pardwa('f',0,0,self.r,self.sigma_p)
                    p.p.color=vec(0,1,0)
                    p.p.visible=True
                    self.pf.append(p)
                    self.niedojrzale_pardwy.append(p)
                self.zlozone_jaja.remove(i)
             else:
                i[0]+=1

    def sprawdz_dojrzalosc_pardw(self):
        for i in self.niedojrzale_pardwy[:]:
            if i.wiek==15:
                i.dojrzalosc()
                self.niedojrzale_pardwy.remove(i)
                #print('urosla nowa pardwa')

    def pardwy_uciekaja(self):
        wilki=self.wm+self.wf
        
        for j in wilki:
            if j.najedzony==False:
                #print('glodny wilk')
                for i in self.pm[:]:
            
                    d=(i.p.pos.x-j.w.pos.x)**2+(i.p.pos.y-j.w.pos.y)**2
                    d=math.sqrt(d)
                
                    if d<self.hpw:
                        if d<self.hpw/10:
                            if i.dojrzala==True:
                                self.nakarm(j) #wilk karmi inne wilki bo pardwa dojrzala
                            j.zjadl() #wilk karmi siebie
                            i.p.visible=False
                            self.pm.remove(i)# pardwa znika
                        else:
                            i.uciekaj_przed_wilkiem(j.w.pos.x,j.w.pos.y)

        for j in wilki:
            if j.najedzony==False:
                #print('glodny wilk')
                for i in self.pf[:]:
                    
                    d=(i.p.pos.x-j.w.pos.x)**2+(i.p.pos.y-j.w.pos.y)**2
                    d=math.sqrt(d)
                    if d<self.hpw:
                        if d<self.hpw/10:
                        
                            if i.dojrzala==True:
                                self.nakarm(j)
                            i.p.visible=False
                            self.pf.remove(i)
                            j.zjadl()
                        
                        else:
                            i.uciekaj_przed_wilkiem(j.w.pos.x,j.w.pos.y)
                        
    def gon_pardwy(self):
        wilki=self.wm+self.wf
        pardwy=self.pf+self.pm
        for i in wilki:
            if i.najedzony==False:
                for j in pardwy:
                    d=(j.p.pos.x-i.w.pos.x)**2+(j.p.pos.y-i.w.pos.y)**2
                    d=math.sqrt(d)
                    i.gon_pardwe(j.p.pos.x,j.p.pos.y,d)
                
    def nakarm(self,wilk):
         wilki=self.wm+self.wf
         for i in wilk.mama: #karmi swoje młode
            i.zjadl()
            
         for i in wilki: #karmi kumpli w promieniu
             d=(i.w.pos.x-wilk.w.pos.x)**2+(i.w.pos.y-wilk.w.pos.y)**2
             d=math.sqrt(d)
             if d<=self.hbreed:
                 i.zjadl()
        
        
   
    def wilki_w_stadzie(self): #dopisane ze alfa sie nie rusza-> klasa wilk
        wilki=self.wm+self.wf
        macierz_spr_wilkow=np.zeros((len(wilki),len(wilki))) #zeby dwa razy nie sprawdzac tych samych pardw w jednej jednsotce czasu
        for i in range(0,len(wilki)): #odleglosci miedzy samcami pardw
            for j in range(0,len(wilki)):
                if i==j:
                    pass
                
                elif wilki[i].alfa==True and wilki[j].alfa==True and i!=j and macierz_spr_wilkow[i][j]==0 and macierz_spr_wilkow[j][i]==0: #intereakcja miedzy alfami
                    if wilki[i].plec!=wilki[j].plec:
                        d=(wilki[i].w.pos.x-wilki[j].w.pos.x)**2+(wilki[i].w.pos.y-wilki[j].w.pos.y)**2
                        d=math.sqrt(d)
                        if d<self.hwalfa:
                            
                            wilki[i].drugi_alfa_blisko(wilki[j].w.pos.x,wilki[j].w.pos.y)
                            wilki[j].drugi_alfa_blisko(wilki[i].w.pos.x,wilki[i].w.pos.y)
                            
                            macierz_spr_wilkow[i][j]=1
                            macierz_spr_wilkow[j][i]=1
                            
                            
                        
                else: #trzymanie sie stada
                    if macierz_spr_wilkow[i][j]==0 and macierz_spr_wilkow[j][i]==0:
                        d=(wilki[i].w.pos.x-wilki[j].w.pos.x)**2+(wilki[i].w.pos.y-wilki[j].w.pos.y)**2
                        d=math.sqrt(d)
                        if d<self.hwmm:
                            wilki[i].trzymaj_sie_stada(wilki[j].w.pos.x,wilki[j].w.pos.y,d)
                            wilki[j].trzymaj_sie_stada(wilki[i].w.pos.x,wilki[i].w.pos.y,d)
                            
                            macierz_spr_wilkow[i][j]=1
                            macierz_spr_wilkow[j][i]=1
            
    def wilki_daza_do_alfy(self):
        wilki=self.wm+self.wf
        for i in range(0,len(wilki)):
            if wilki[i].alfa==True:
                for j in range(0,len(wilki)):
                    if i!=j and wilki[j].alfa!=True:
                        wilki[j].dazy_do_alfa(wilki[i].w.pos.x,wilki[j].w.pos.y)

    def gody_wilkow(self):
        
        for i in range(0,len(self.wm)):
            for j in range(0,len(self.wf)):
                    
                if self.wm[i].dojrzala==True and self.wf[j].dojrzala==True and self.wf[j].ciaza==False:
                
                    d=(self.wm[i].w.pos.x-self.wf[j].w.pos.x)**2+(self.wm[i].w.pos.y-self.wf[j].w.pos.y)**2
                    d=math.sqrt(d)
                
                    if d<self.hwfm:
                        self.wf[j].po_godach(self.wm[i].w.pos.x,self.wm[i].w.pos.y)
                        self.wm[i].po_godach(self.wf[j].w.pos.x,self.wf[j].w.pos.y)
                        
                        self.po_godach_wilkow.append([self.wm[i],self.wf[j],10])
                    else:
                        self.wm[i].gody_wilkow(self.wf[j].w.pos.x,self.wf[j].w.pos.y)

    def po_godach_wilkow_funkcja(self):
        for i in self.po_godach_wilkow[:]:

            if i[2]>2:
                i[0].po_godach(i[1].w.pos.x,i[1].w.pos.y)
                i[1].po_godach(i[0].w.pos.x,i[0].w.pos.y)
                i[2]=i[2]-1
                                                
            elif i[2]==0:
                i[1].ciaza=True
                self.w_ciazy.append([i[1],False])
                self.po_godach_wilkow.remove(i)
            else:
                i[2]=i[2]-1
                
    def wilki_do_brzucha(self):
        for i in self.w_ciazy[:]:
            
            if i[1]==False:
                i[1]=i[0].ile_wilkow_bedzie()
             #wiemy ile juz jaj zlozy
            if i[1]>0:
                self.wilki_w_brzuchu.append([i[0]])  #->i[0] mama wilka
                i[1]=i[1]-1
            if i[1]==0:
                i[0].ciaza=False
                self.w_ciazy.remove(i)

    def narodziny_wilkow(self):
        for i in self.wilki_w_brzuchu[:]:
                plec_wilka=rn.uniform(0,1)
                polozenie=i[0].w.pos
                if plec_wilka<=0.5:
                    plec_wilka='m'
                    w=wilk('m',polozenie.x,polozenie.y,self.r,self.sigma_w,self.na_glodzie)
                    w.w.visible=True
                    self.wm.append(w)
                    self.niedojrzale_wilki.append([w,i[0]])
                else:
                    plec_wilka='f'
                    w=wilk('f',polozenie.x,polozenie.y,self.r,self.sigma_w,self.na_glodzie)
                    #w=wilk('f',0,0,self.r,self.sigma_w)
                    w.w.visible=True
                    self.wf.append(w)
                    self.niedojrzale_wilki.append([w,i[0]])
                i[0].mama.append(w)
                self.wilki_w_brzuchu.remove(i)
                #print('narodziny wilka')
                
    def sprawdz_dojrzalosc_wilkow(self):
        for i in self.niedojrzale_wilki[:]:
            if i[0].wiek==30:
                i[0].dojrzalosc()
                if i[1]!=False:
                    i[1].mama.remove(i[0])
                self.niedojrzale_wilki.remove(i)
            else:
                if i[1]!=False:
                    i[0].mlode_trzymaj_sie_mamy(i[1].w.pos.x,i[1].w.pos.y)
                    i[0].przemieszczenie=True
    
    def zauktualizuj_czy_wilki_sa_glodne(self):
        wilki=self.wm+self.wf
        for i in wilki:
            i.sprawdz_czy_najedzony()
        
    def brown(self): #brown dla wszystkich
        for i in self.pf:
            i.p.make_trail=True
            i.brown()
        for i in self.pm:
            i.p.make_trail=True
            i.brown()
        for i in self.wf:
            i.w.make_trail=True
            i.brown()
        for i in self.wm:
            i.w.make_trail=True
            i.brown()
            
    def starzenie(self):
        self.pf=[i for i in self.pf if i.smierc() !=True]
        self.pm=[i for i in self.pm if i.smierc() !=True]
        self.wm=[i for i in self.wm if i.smierc() !=True]
        self.wf=[i for i in self.wf if i.smierc() !=True]
        #usuwanie z innyhc kulbów jeśli jakies jest martwe
        wilki=self.wm+self.wf
        pardwy=self.pf+self.pm
        self.niedojrzale_pardwy=[i for i in self.niedojrzale_pardwy if i in pardwy]
        self.skladajace_jaja=[i for i in self.skladajace_jaja if i in pardwy]
        self.niedojrzale_wilki=[i for i in self.niedojrzale_wilki if i[0] in wilki]
        self.w_ciazy=[i for i in self.w_ciazy if i[0] in wilki]
        
        for i in self.niedojrzale_wilki[:]:
            if i[1]==False:
                pass
            else:
                if i[1].wiek<=i[1].czas_zycia:
                    pass
                else:
                    i[1]=False
                

        self.lpf=len(self.pf)
        self.lpm=len(self.pm)
        self.lwm=len(self.wm)
        self.lwf=len(self.wf)

    def nie_przemieszczone_przemiescic(self):
        for i in self.pf:
            i.nie_przemieszczona_p_przemiescic()
        for i in self.pm:
            i.nie_przemieszczona_p_przemiescic()
        for i in self.wf:
            i.nie_przemieszczona_w_przemiescic()
        for i in self.wm:
            i.nie_przemieszczona_w_przemiescic()

    def kasuj_przemieszczenia(self):
        for i in self.pf:
            i.kasuj_przemieszczenia()
        for i in self.pm:
            i.kasuj_przemieszczenia()
        for i in self.wf:
            i.kasuj_przemieszczenia()
        for i in self.wm:
            i.kasuj_przemieszczenia()

In [41]:
okno=Tk()
okno.title('MÓJ WŁASNY LAS')
okno.configure(background="lightblue")
frame=Frame(okno)

czcionka=8
c=czcionka
c2=7
wid=4

et=Label(okno,text="Number of female rabbits",font=('Comic Sans MS',c2,'bold'))
et.grid(row=0,column=0)
et.configure(background="lightblue",fg='blue')

lpf = IntVar(okno, value=200)
LPF=Entry(okno,textvariable=lpf,width=wid)
LPF.configure(font=('Comic Sans MS',c))
LPF.grid(row=0,column=1)

et2=Label(okno,text="Number of male rabbits",font=('Comic Sans MS',c2,'bold'))
et2.grid(row=0,column=2)
et2.configure(background="lightblue",fg='blue')

lpm = IntVar(okno, value=200)
LPM=Entry(okno,textvariable=lpm, width=wid)
LPM.configure(font=('Comic Sans MS',c))
LPM.grid(row=0,column=3)

et3=Label(okno,text="Number of female wolves",font=('Comic Sans MS',c2,'bold'))
et3.grid(row=1,column=0)
et3.configure(background="lightblue",fg='blue')

lwf = IntVar(okno, value=0)
LWF=Entry(okno,textvariable=lwf, width=wid)
LWF.configure(font=('Comic Sans MS',c))
LWF.grid(row=1,column=1)

et3=Label(okno,text="Number of male wolves",font=('Comic Sans MS',c2,'bold'))
et3.grid(row=1,column=2)
et3.configure(background="lightblue",fg='blue')

lwm = IntVar(okno, value=0)
LWM=Entry(okno,textvariable=lwm, width=wid)
LWM.configure(font=('Comic Sans MS',c))
LWM.grid(row=1,column=3)

et4=Label(okno,text="Size of the animal",font=('Comic Sans MS',c2,'bold'))
et4.grid(row=2,column=0)
et4.configure(background="lightblue",fg='blue')

R=IntVar(okno, value=500)
RR=Entry(okno,textvariable=R, width=wid)
RR.configure(font=('Comic Sans MS',c))
RR.grid(row=2,column=1)

et5=Label(okno,text="Parameter 1: MaleMaleReactionRabbit",font=('Comic Sans MS',c2,'bold'))
et5.grid(row=2,column=2)
et5.configure(background="lightblue",fg='blue')


hpmm=IntVar(okno, value=30) #dopuszczalna odległość między samcami pardw
HPMM=Entry(okno,textvariable=hpmm, width=wid)
HPMM.configure(font=('Comic Sans MS',c))
HPMM.grid(row=2,column=3)

et6=Label(okno,text="Parameter 2: MaleFemaleReactionRabbit",font=('Comic Sans MS',c2,'bold'))
et6.grid(row=3,column=0)
et6.configure(background="lightblue",fg='blue')

hpfm=IntVar(okno, value=350) #dopuszczalna odległość między samiec samica pardw
HPFM=Entry(okno,textvariable=hpfm, width=wid)
HPFM.configure(font=('Comic Sans MS',c))
HPFM.grid(row=3,column=1)

et7=Label(okno,text="Parameter 3: WolfRabbitReaction",font=('Comic Sans MS',c2,'bold'))
et7.grid(row=3,column=2)
et7.configure(background="lightblue",fg='blue')

hpw=IntVar(okno, value=1) #dopuszczalana odległość pardwa wilk
HPW=Entry(okno,textvariable=hpw, width=wid)
HPW.configure(font=('Comic Sans MS',c))
HPW.grid(row=3,column=3)

et8=Label(okno,text="Parameter 4: MaleMaleReactionWolf",font=('Comic Sans MS',c2,'bold'))
et8.grid(row=4,column=0)
et8.configure(background="lightblue",fg='blue')

hwmm=IntVar(okno, value=1) #dopuszczalana odległość między samcami wilków
HWMM=Entry(okno,textvariable=hwmm, width=wid)
HWMM.configure(font=('Comic Sans MS',c))
HWMM.grid(row=4,column=1)

et9=Label(okno,text="Parameter 5: FemaleMaleReactionWolf",font=('Comic Sans MS',c2,'bold'))
et9.grid(row=4,column=2)
et9.configure(background="lightblue",fg='blue')

hwfm=IntVar(okno, value=1) #odległośc samiec samica wilka
HWFM=Entry(okno,textvariable=hwfm, width=wid)
HWFM.configure(font=('Comic Sans MS',c))
HWFM.grid(row=4,column=3)

e0=Label(okno,text="Alpha individual impact",font=('Comic Sans MS',c2,'bold'))
e0.grid(row=9,column=0)
e0.configure(background="lightblue",fg='blue')

hwalfa=IntVar(okno, value=1) #odległośc samiec samica wilka
HWALFA=Entry(okno,textvariable=hwalfa, width=wid)
HWALFA.configure(font=('Comic Sans MS',c))
HWALFA.grid(row=9,column=1)

e1=Label(okno,text="Breed ratio",font=('Comic Sans MS',c2,'bold'))
e1.grid(row=9,column=2)
e1.configure(background="lightblue",fg='blue')

hbreed=IntVar(okno, value=1) #odległośc samiec samica wilka
HBREED=Entry(okno,textvariable=hbreed, width=wid)
HBREED.configure(font=('Comic Sans MS',c))
HBREED.grid(row=9,column=3)


e2=Label(okno,text="Time wolf is full",font=('Comic Sans MS',c2,'bold'))
e2.grid(row=10,column=0)
e2.configure(background="lightblue",fg='blue')

czas_na_glodzie=IntVar(okno, value=1) #odległośc samiec samica wilka
CZAS_NA_GLODZIE=Entry(okno,textvariable=czas_na_glodzie, width=wid)
CZAS_NA_GLODZIE.configure(font=('Comic Sans MS',c))
CZAS_NA_GLODZIE.grid(row=10,column=1)



LPM.focus()
LPF.focus()
LWM.focus()
LWF.focus()
RR.focus()
HPFM.focus()
HPW.focus()
HWMM.focus()
HWFM.focus()
HWALFA.focus()
CZAS_NA_GLODZIE.focus()
HBREED.focus()


#legenda
et9=Label(okno,text="immature rabbits",font=('Comic Sans MS',c2,'bold'))
et9.grid(row=8,column=0)
et9.configure(background="lightblue",fg='blue')

et10=Label(okno,text="",font=('Comic Sans MS',c2,'bold'))
et10.grid(row=8,column=1)
et10.configure(background='green2',fg='green2',width=5)

et999=Label(okno,text="immature wolves",font=('Comic Sans MS',c2,'bold'))
et999.grid(row=8,column=2)
et999.configure(background="lightblue",fg='blue')

et100=Label(okno,text="",font=('Comic Sans MS',c2,'bold'))
et100.grid(row=8,column=3)
et100.configure(background='yellow',fg='yellow',width=5)

et11=Label(okno,text="rabbits",font=('Comic Sans MS',c2,'bold'))
et11.grid(row=7,column=0)
et11.configure(background="lightblue",fg='blue')

et111=Label(okno,text="rabbits",font=('Comic Sans MS',c2,'bold'))
et111.grid(row=7,column=1)
et111.configure(background="red",fg='red',width=5)

et12=Label(okno,text="wolves",font=('Comic Sans MS',c2,'bold'))
et12.grid(row=7,column=2)
et12.configure(background="lightblue",fg='blue')

et13=Label(okno,text="",font=('Comic Sans MS',c2,'bold'))
et13.grid(row=7,column=3)
et13.configure(background="blue",fg='blue',width=5)

et14=Label(okno,text="female rabbits",font=('Comic Sans MS',c2,'bold'))
et14.grid(row=5,column=0)
et14.configure(background="lightblue",fg='blue')

et15=Label(okno,text="",font=('Comic Sans MS',c2,'bold'))
et15.grid(row=5,column=1)
et15.configure(background="orchid3",fg='orchid3',width=5)

et16=Label(okno,text="male rabbits",font=('Comic Sans MS',c2,'bold'))
et16.grid(row=5,column=2)
et16.configure(background="lightblue",fg='blue')

et17=Label(okno,text="",font=('Comic Sans MS',c2,'bold'))
et17.grid(row=5,column=3)
et17.configure(background="purple4",fg='purple4',width=5)

et18=Label(okno,text="female wolves",font=('Comic Sans MS',c2,'bold'))
et18.grid(row=6,column=0)
et18.configure(background="lightblue",fg='blue')

et19=Label(okno,text="",font=('Comic Sans MS',c2,'bold'))
et19.grid(row=6,column=1)
et19.configure(background="gray77",fg='gray77',width=5)

et20=Label(okno,text="male wolves",font=('Comic Sans MS',c2,'bold'))
et20.grid(row=6,column=2)
et20.configure(background="lightblue",fg='blue')

et21=Label(okno,text="",font=('Comic Sans MS',c2,'bold'))
et21.grid(row=6,column=3)
et21.configure(background="black",fg='black',width=5)



###########################################ZMIENNE###########

global moj_las
global idx #index do odmierzania czasu
running = True
global dt
sigma_p=30 #przy ruchu browna współczynnik do zmieniania
sigma_w=4
idx=0 #sigma w lesie



###########FUNKCJE ZEBY ANIMACJA TRWAŁA CAŁY CZAS##############
def start():
    global running
    running = True
    global idx
    idx=0
def stop():
    global running
    running = False

######################### FUNKCJA USTAWIAJĄCA WSYZTSKIE PARAMETRY #########
def run1(): #ustaw las
    global moj_las
    start()
    moj_las=las(lpf.get(),lpm.get(),lwf.get(),lwm.get(),R.get(),10,sigma_p,sigma_w,hpmm.get(),hpfm.get(),hpw.get(),hwmm.get(),hwfm.get(),hwalfa.get(),hbreed.get(),czas_na_glodzie.get())
    """for i in moj_las.pf:
        i.p.make_trail=True
    for i in moj_las.pm:
        i.p.make_trail=True
    for i in moj_las.wm:
        i.w.make_trail=True
    for i in moj_las.wf:
        i.w.make_trail=True"""
############ PUSZCZENIE ANIMACJI I SCZYTYWANIE Z NIEJ DANYCH ############


global pf,pm,wm,wf

pf=np.array([0]) #pf- ile pardw żeńskich, pm ile męskich, wf ile wilków żeńskich, wm ile męskich
pm=np.array([0])
wm=np.array([0])
wf=np.array([0])
p=np.array([0]) #liczba pardw ogółem
w=np.array([0])
niedojrzale_pf=np.array([0])
niedojrzale_wilki=np.array([0])

#Do wykresu:
global x
x=np.array([idx])#momenty czasowe do rysowania wykresu ile kogo w lesie jest
plt.ion()

fig = Figure(figsize=(5,5))

ax = fig.add_subplot(311)
ax2= fig.add_subplot(312)
ax3= fig.add_subplot(313)

legend_size=4

ax.plot(x,pf,color=(0.7,0.4,1),label='pardwy żeńskie')
ax.plot(x,pm,color=(0.49,0,1),label='pardwy męskie')
ax.plot(x,wf,color=(0.75,0.75,0.75),label='wilki żeńskie')
ax.plot(x,wm,color='black',label='wilki męskie')
ax.tick_params(axis="y", labelsize=7) 


ax2.plot(x,p,color='red',label='pardwy')
ax2.plot(x,w,color='blue',label='wilki')
ax2.tick_params(axis="y", labelsize=7) 

ax3.plot(x,niedojrzale_pf,color=(0,1,0),label='pardwy')
ax3.plot(x,niedojrzale_wilki,color='yellow',label='wilki')
ax3.tick_params(axis="y", labelsize=7) 

ax.set_title ("Population size: Female-Male ", fontsize=10)
ax.set_ylabel("Size", fontsize=10)
ax.set_xlabel("Time", fontsize=10)

ax2.set_title ("Population size: Wolves-Rabbits", fontsize=10)
ax2.set_ylabel("Size", fontsize=8)
ax2.set_xlabel("Time", fontsize=8)

ax3.set_title ("Population size: Immature individuals", fontsize=10)
ax3.set_ylabel("Size", fontsize=8)
ax3.set_xlabel("Time", fontsize=8)

"""ax.legend()
ax2.legend()
ax3.legend()"""

fig.subplots_adjust(wspace=1, hspace=1)

okno.canvas = FigureCanvasTkAgg(fig,master=okno)
okno.canvas.show()
okno.canvas.get_tk_widget().grid(row=15,column=0)
frame.grid(row=20,column=0)
okno.canvas.draw()
plt.show()

#koniec do wykresu

#FUNKCJA DO GUZICZKA
def run(): #pusc las->main loop gdy klikniemy zeby puśić animajcę 
    global idx
    global moj_las
    global x
    global pf
    pf=np.array([len(moj_las.pf)])
    pm=np.array([moj_las.lpm])
    wf=np.array([moj_las.lwf])
    wm=np.array([moj_las.lwm])
    w=np.array([moj_las.lwm+moj_las.lwf])
    p=np.array([moj_las.lpf+moj_las.lpm])
    niedojrzale_pardwy=np.array([len(moj_las.niedojrzale_pardwy)])
    niedojrzale_wilki=np.array([len(moj_las.niedojrzale_wilki)])
    while True:
        if idx%20==0: #aktualizowanie
            pf=np.append(pf,moj_las.lpf) #to do wykresu
            pm=np.append(pm,moj_las.lpm)
            wf=np.append(wf,moj_las.lwf)
            wm=np.append(wm,moj_las.lwm)
            niedojrzale_pardwy=np.append(niedojrzale_pardwy,len(moj_las.niedojrzale_pardwy))
            niedojrzale_wilki=np.append(niedojrzale_wilki,len(moj_las.niedojrzale_wilki))
            x=np.append(x,idx)

            w=np.append(w,moj_las.lwm+moj_las.lwf)
            p=np.append(p,moj_las.lpf+moj_las.lpm)
            """print('p',p)
            print('x',x)
            print('niedojrzale_pardwy',niedojrzale_pardwy)
            print('pf',pf)
            print('pm',pm)"""
            
            okno.update()
            
            ax.plot(x,pf,color=(0.7,0.4,1),label='pardwy żeńskie')
            ax.plot(x,pm,color=(0.49,0,1),label='pardwy męskie')
            ax.plot(x,wf,color=(0.75,0.75,0.75),label='wilki żeńskie')
            ax.plot(x,wm,color='black',label='wilki męskie')
            ax2.plot(x,p,color='red',label='pardwy')
            ax2.plot(x,w,color='blue',label='wilki')
            ax3.plot(x,niedojrzale_pardwy,color=(0,1,0),label='pardwy')
            ax3.plot(x,niedojrzale_wilki,color='yellow',label='wilki')
            okno.canvas.draw()

        if running:
            #dane do wykresu
            pf=np.append(pf,moj_las.lpf) #to do wykresu
            pm=np.append(pm,moj_las.lpm)
            wf=np.append(wf,moj_las.lwf)
            wm=np.append(wm,moj_las.lwm)
            niedojrzale_pardwy=np.append(niedojrzale_pardwy,len(moj_las.niedojrzale_pardwy))
            niedojrzale_wilki=np.append(niedojrzale_wilki,len(moj_las.niedojrzale_wilki))
            x=np.append(x,idx)

            w=np.append(w,moj_las.lwm+moj_las.lwf)
            p=np.append(p,moj_las.lpf+moj_las.lpm)
            
            moj_las.run() #przesuwanie zwierzątek
            idx+=1
            rate(100)
        else:
            break
            
    

guzik=Button(okno,text='PREPARE SIMULATION',font=('Comic Sans MS',c2,'bold'),bg='white',fg='red',command=lambda: run1())
guzik.grid(row=11,column=0)

guzik2=Button(okno,text='RUN',font=('Comic Sans MS',c2,'bold'),bg='white',fg='red',command=lambda: run())
guzik2.grid(row=11,column=1)

stop = Button(okno, text="STOP", command=stop,font=('Comic Sans MS',c2,'bold'),bg='white',fg='red').grid(row=11,column=2)
mainloop()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:309: MatplotlibDeprecationWarning: The FigureCanvasTk.show function was deprecated in version 2.2. Use FigureCanvasTk.draw instead.


<IPython.core.display.Javascript object>